## Import Libraries

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## Import Dataset

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Rescales all images by 1/255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_dir = '../input/aid-mini/train'
test_dir = '../input/aid-mini/test'
# train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,              # Target directory
    target_size=(150, 150), # Resizes all images to 150 × 150
    batch_size=20,
    class_mode='categorical')

# validation generator
validation_generator = test_datagen.flow_from_directory(
    test_dir,         # Target directory
    target_size=(150, 150), # Resizes all images to 150 × 150
    batch_size=20,
    class_mode='categorical')

Found 1200 images belonging to 30 classes.
Found 600 images belonging to 30 classes.


## Without Augmentation

### Random Weight Initialization 

In [ ]:
from keras.applications.xception import Xception
conv_base = Xception(include_top=False,
                  input_shape=(150, 150, 3),
                  weights=None)

conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 74, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 74, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(conv_base)
model.add(keras.layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(30, activation='softmax'))

# model.summary()

In [ ]:
model.build(input_shape=(None,150, 150, 3))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1536030   
Total params: 22,397,510
Trainable params: 22,342,982
Non-trainable params: 54,528
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_generator,
                    epochs=100,
                    validation_data=validation_generator
                    )

Epoch 1/100
60/60 [==============================] - 16s 182ms/step - loss: 3.2623 - acc: 0.0793 - val_loss: 3.4012 - val_acc: 0.0333
Epoch 2/100
60/60 [==============================] - 10s 158ms/step - loss: 2.1452 - acc: 0.5502 - val_loss: 3.4014 - val_acc: 0.0333
Epoch 3/100
60/60 [==============================] - 10s 159ms/step - loss: 1.5785 - acc: 0.7265 - val_loss: 3.4020 - val_acc: 0.0333
Epoch 4/100
60/60 [==============================] - 10s 161ms/step - loss: 1.1373 - acc: 0.8670 - val_loss: 3.4032 - val_acc: 0.0333
Epoch 5/100
60/60 [==============================] - 10s 158ms/step - loss: 0.8430 - acc: 0.9003 - val_loss: 3.4061 - val_acc: 0.0333
Epoch 6/100
60/60 [==============================] - 10s 158ms/step - loss: 0.6612 - acc: 0.9326 - val_loss: 3.4103 - val_acc: 0.0350
Epoch 7/100
60/60 [==============================] - 10s 159ms/step - loss: 0.4646 - acc: 0.9577 - val_loss: 3.4174 - val_acc: 0.0383
Epoch 8/100
60/60 [==============================] - 10s 164ms

### Feature Extraction Using ImageNet

In [ ]:

conv_base = Xception(include_top=False,
                  input_shape=(150, 150, 3),
                  weights='imagenet')

conv_base.summary()

83689472/83683744 [==============================] - 2s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 74, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 74, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 74, 32)   0           block1_conv1_bn[0][0]            
__________________________

In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(conv_base)
model.add(keras.layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(30, activation='softmax'))

# model.summary()

In [ ]:
conv_base.trainable = False



In [ ]:
model.build(input_shape=(None,150, 150, 3))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1536030   
Total params: 22,397,510
Trainable params: 1,536,030
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_generator,
                    epochs=100,
                    validation_data=validation_generator
                    )

Epoch 1/100
60/60 [==============================] - 9s 108ms/step - loss: 3.2242 - acc: 0.1384 - val_loss: 2.3407 - val_acc: 0.4000
Epoch 2/100
60/60 [==============================] - 5s 87ms/step - loss: 1.8078 - acc: 0.6414 - val_loss: 1.8634 - val_acc: 0.5483
Epoch 3/100
60/60 [==============================] - 6s 99ms/step - loss: 1.2273 - acc: 0.8086 - val_loss: 1.6339 - val_acc: 0.6000
Epoch 4/100
60/60 [==============================] - 5s 86ms/step - loss: 0.9020 - acc: 0.8754 - val_loss: 1.4849 - val_acc: 0.6450
Epoch 5/100
60/60 [==============================] - 5s 91ms/step - loss: 0.7272 - acc: 0.9107 - val_loss: 1.4087 - val_acc: 0.6467
Epoch 6/100
60/60 [==============================] - 5s 88ms/step - loss: 0.5831 - acc: 0.9427 - val_loss: 1.3417 - val_acc: 0.6683
Epoch 7/100
60/60 [==============================] - 5s 90ms/step - loss: 0.4918 - acc: 0.9637 - val_loss: 1.2976 - val_acc: 0.6833
Epoch 8/100
60/60 [==============================] - 5s 88ms/step - loss: 0

### Fine Tuning

In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block12_sepconv1':
    set_trainable = True

  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1536030   
Total params: 22,397,510
Trainable params: 9,938,390
Non-trainable params: 12,459,120
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_generator,
                    epochs=100,
                    validation_data=validation_generator
                    )

Epoch 1/100
60/60 [==============================] - 12s 149ms/step - loss: 1.0952 - acc: 0.7945 - val_loss: 1.2940 - val_acc: 0.6917
Epoch 2/100
60/60 [==============================] - 7s 119ms/step - loss: 0.3281 - acc: 0.9530 - val_loss: 1.2859 - val_acc: 0.7100
Epoch 3/100
60/60 [==============================] - 7s 121ms/step - loss: 0.1685 - acc: 0.9887 - val_loss: 1.3058 - val_acc: 0.7067
Epoch 4/100
60/60 [==============================] - 5s 90ms/step - loss: 0.0901 - acc: 0.9969 - val_loss: 1.2602 - val_acc: 0.7100
Epoch 5/100
60/60 [==============================] - 6s 101ms/step - loss: 0.0574 - acc: 0.9984 - val_loss: 1.1995 - val_acc: 0.7150
Epoch 6/100
60/60 [==============================] - 5s 91ms/step - loss: 0.0469 - acc: 1.0000 - val_loss: 1.1424 - val_acc: 0.7150
Epoch 7/100
60/60 [==============================] - 6s 94ms/step - loss: 0.0331 - acc: 0.9983 - val_loss: 1.1190 - val_acc: 0.7167
Epoch 8/100
60/60 [==============================] - 5s 91ms/step - los

## With Data Augmentation

In [ ]:
# train generator
from tensorflow.keras.applications.xception import preprocess_input
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.5,1.5],
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical')

# validation generator
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical')

Found 1200 images belonging to 30 classes.
Found 600 images belonging to 30 classes.


### Random Weight Initialization

In [ ]:
from keras.applications.xception import Xception
conv_base = Xception(include_top=False,
                  input_shape=(150, 150, 3),
                  weights=None)

conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 74, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 74, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 74, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(conv_base)
model.add(keras.layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(30, activation='softmax'))

# model.summary()

In [ ]:
model.build(input_shape=(None,150, 150, 3))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
flatten_3 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 30)                1536030   
Total params: 22,397,510
Trainable params: 22,342,982
Non-trainable params: 54,528
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_generator,
                    epochs=100,
                    validation_data=validation_generator
                    )

Epoch 1/100
60/60 [==============================] - 22s 283ms/step - loss: 3.4258 - acc: 0.0544 - val_loss: 3.4012 - val_acc: 0.0333
Epoch 2/100
60/60 [==============================] - 15s 252ms/step - loss: 2.9634 - acc: 0.1929 - val_loss: 3.4012 - val_acc: 0.0333
Epoch 3/100
60/60 [==============================] - 16s 259ms/step - loss: 2.6543 - acc: 0.2764 - val_loss: 3.4014 - val_acc: 0.0333
Epoch 4/100
60/60 [==============================] - 15s 246ms/step - loss: 2.4360 - acc: 0.3181 - val_loss: 3.4015 - val_acc: 0.0333
Epoch 5/100
60/60 [==============================] - 15s 256ms/step - loss: 2.2736 - acc: 0.3323 - val_loss: 3.4017 - val_acc: 0.0333
Epoch 6/100
60/60 [==============================] - 15s 250ms/step - loss: 2.1362 - acc: 0.3852 - val_loss: 3.4013 - val_acc: 0.0333
Epoch 7/100
60/60 [==============================] - 15s 255ms/step - loss: 2.0019 - acc: 0.4072 - val_loss: 3.3957 - val_acc: 0.0333
Epoch 8/100
60/60 [==============================] - 15s 250ms

### Feature Extraction Using ImageNet

In [ ]:

conv_base = Xception(include_top=False,
                  input_shape=(150, 150, 3),
                  weights='imagenet')

conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 74, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 74, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 74, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(conv_base)
model.add(keras.layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(30, activation='softmax'))



In [ ]:
conv_base.trainable = False



In [ ]:
model.build(input_shape=(None,150, 150, 3))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
flatten_4 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 30)                1536030   
Total params: 22,397,510
Trainable params: 1,536,030
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_generator,
                    epochs=100,
                    validation_data=validation_generator
                    )

Epoch 1/100
60/60 [==============================] - 17s 238ms/step - loss: 3.4144 - acc: 0.1106 - val_loss: 2.5361 - val_acc: 0.3350
Epoch 2/100
60/60 [==============================] - 13s 210ms/step - loss: 2.1618 - acc: 0.4203 - val_loss: 2.0656 - val_acc: 0.4683
Epoch 3/100
60/60 [==============================] - 12s 208ms/step - loss: 1.6589 - acc: 0.5925 - val_loss: 1.8199 - val_acc: 0.5350
Epoch 4/100
60/60 [==============================] - 13s 219ms/step - loss: 1.3833 - acc: 0.6527 - val_loss: 1.6567 - val_acc: 0.5783
Epoch 5/100
60/60 [==============================] - 12s 205ms/step - loss: 1.2587 - acc: 0.6746 - val_loss: 1.5371 - val_acc: 0.6117
Epoch 6/100
60/60 [==============================] - 12s 206ms/step - loss: 1.0187 - acc: 0.7456 - val_loss: 1.4710 - val_acc: 0.6200
Epoch 7/100
60/60 [==============================] - 13s 221ms/step - loss: 0.9428 - acc: 0.7575 - val_loss: 1.4015 - val_acc: 0.6517
Epoch 8/100
60/60 [==============================] - 13s 209ms

In [ ]:

# conv_base = Xception(include_top=False,
#                   input_shape=(150, 150, 3),
#                   weights='imagenet')

# conv_base.summary()

### Fine Tuning

In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block12_sepconv1':
    set_trainable = True

  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
flatten_4 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 30)                1536030   
Total params: 22,397,510
Trainable params: 9,938,390
Non-trainable params: 12,459,120
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_generator,
                    epochs=100,
                    validation_data=validation_generator
                    )

Epoch 1/100
60/60 [==============================] - 19s 252ms/step - loss: 1.3545 - acc: 0.7064 - val_loss: 1.0718 - val_acc: 0.7167
Epoch 2/100
60/60 [==============================] - 13s 213ms/step - loss: 0.8309 - acc: 0.8333 - val_loss: 1.0572 - val_acc: 0.7367
Epoch 3/100
60/60 [==============================] - 13s 225ms/step - loss: 0.6589 - acc: 0.8542 - val_loss: 0.9857 - val_acc: 0.7350
Epoch 4/100
60/60 [==============================] - 13s 215ms/step - loss: 0.4729 - acc: 0.9150 - val_loss: 0.9162 - val_acc: 0.7550
Epoch 5/100
60/60 [==============================] - 13s 212ms/step - loss: 0.4748 - acc: 0.8772 - val_loss: 0.8756 - val_acc: 0.7583
Epoch 6/100
60/60 [==============================] - 13s 221ms/step - loss: 0.3297 - acc: 0.9290 - val_loss: 0.8783 - val_acc: 0.7550
Epoch 7/100
60/60 [==============================] - 13s 217ms/step - loss: 0.3248 - acc: 0.9151 - val_loss: 0.8320 - val_acc: 0.7717
Epoch 8/100
60/60 [==============================] - 13s 219ms